# Geospatial data

Python supports many tools for working with geospatial data. Some are wrappers around well-known libraries like `pyproj` that wraps the `proj.4`-library. Others provide building blocks, like `shapely` which gives us objects like points, lines and polygons.

## Geopandas

`geopandas` is package built on top of `pandas` to provide a `GeoDataframe`. In a `GeoDataframe` each data point is associated with a geometry. The geometry can be as simple as a single point, or as complex as a collection of points, lines and polygons. Under the hood `geopandas` uses packages like `pyproj`, `shapely`, `fiona` and others to do the heavy lifting.

Currently `geopandas` is not yet a part of the core Anaconda package. It can still be installed using `conda` though, although from the `conda-forge`-channel. There has been some dependency problems with the `fiona`-package causing some problems with the installation. However, the following seems to work:

    conda install -c conda-forge geopandas icu=58 kealib=1.4.7 matplotlib=2

In [ ]:
%matplotlib notebook

import geopandas as gpd

### Fire stations in Norway

Through `fiona`, `geopandas` supports reading many different formats for geospatial data. The following file contains information about all fire stations in Norway. It was provided by [Direktoratet for samfunnsikkerhet og beredskap (DSB)](https://www.dsb.no) and downloaded from [https://raw.githubusercontent.com/dsb-norge/static-share/master/firestations.json](https://raw.githubusercontent.com/dsb-norge/static-share/master/firestations.json).

In [ ]:
fire = gpd.read_file('data/firestations.json')
fire.head()

Note the column __geometry__. Every `GeoDataframe` has one such geometry column containing the geospatial information. `geopandas` can use this information to plot the dataset, as well as many other spatial operations.

In [ ]:
fire.plot();

We note that there are several data points that have a geometry that is clearly wrong. For this example, we'll simply throw away those data. To do so, we construct a bounding box that contains the data we want to keep, but not the data we want to throw out (which in this example is very easy). To create the bounding box we use the `shapely`-library directly. When we have the bounding box, we simply ask `geopandas` to figure out which points are _within_ the bounding box, and we create a new `GeoDataframe` consisting of only those points.

In [ ]:
from shapely import geometry
bbox = geometry.Polygon([(0, 50), (50, 50), (50, 90), (0, 90)])

fire = fire[fire.within(bbox)]
fire.plot();

One important note: We recognize that the geometry of these data are given as longitudes and latitudes (even though UTM easting and northing is also given as other columns in the dataset). Using `pyproj` `geopandas` knows about different coordinate reference systems. We can query the dataframe about which CRS it uses:

In [ ]:
fire.crs

This will return a dictionary of `proj.4`-parameters. In this case the parameters are simply a reference to an [EPSG-number](https://epsg.io/). Those in the know might recognize that [4326 represents the WGS84 reference system](https://epsg.io/4326).

### Norwegian regions (fylker)

Let us create a second `GeoDataframe`, this one representing the Norwegian administrative regions (fylker). The [Norwegian Mapping Authority](https://kartverket.no/) (Kartverket) makes a lot of data available, and at [Geonorge](https://kartkatalog.geonorge.no/metadata/kartverket/administrative-enheter-fylker/6093c8a8-fa80-11e6-bc64-92361f002671) we can download the borders of the Norwegian regions (fylker).

The data are available in a format called [SOSI](https://en.wikipedia.org/wiki/SOSI). The SOSI format is unfortunately not supported by `geopandas`, so we need to first convert it. A tool called [sosicon](http://sosicon.espenandersen.no/) is available that can convert the SOSI-files to Shapefiles.

+ Download and install `sosicon` from [sosicon.espenandersen.no/download/](http://sosicon.espenandersen.no/download/)
+ Convert the SOSI-file you downloaded. If you use the command line version of `sosicon` the command is

    sosicon -2shp name_of_sosi_file.sos
    
With the data in Shapefile-format we can easily load it into `geopandas`.

In [ ]:
fylker = gpd.read_file('examples/Basisdata_0000_Norge_25833_Fylker_SOSI_Fylke_FLATE.shp')
fylker.head()

In [ ]:
fylker.plot(column='FYLKESNUMM', cmap='tab20'); #, legend=True)

Now, we have two `GeoDataframes`. Let us try to join them, in order to figure out in which region each fire station is located. We use the _spatial join_-function in `geopandas`, and say that the operation we are interested in is `within`:

In [ ]:
gpd.sjoin(fire, fylker[['FYLKESNUMM', 'geometry']], op='within')

Oops. `geopandas` warns us that our two dataframes have different coordinate systems, so it cannot do the spatial join. Let us convert the `fylker`-data to the firestations coordinate reference system, and try the join again:

In [ ]:
fire = gpd.sjoin(fire, fylker[['FYLKESNUMM', 'geometry']].to_crs(fire.crs), op='within')
fire

In [ ]:
fire.plot(column='FYLKESNUMM', cmap='tab20');

In [ ]:
fire.groupby('FYLKESNUMM').mean()

In [ ]:
fire.groupby('FYLKESNUMM').size()

We can now also add the firestation data back to the `fylker`-dataset. In this case, for instance a summary of the number of fire stations. Our `groupby` is done by the region number, so for simplicity we'll say that the region number is the index of `fylker`.

In [ ]:
fylker.head()

In [ ]:
fylker = fylker.set_index('FYLKESNUMM', drop=False)
fylker.head()

In [ ]:
fylker['ant_brannst'] = fire.groupby('FYLKESNUMM').size()
fylker.head()

In [ ]:
fylker.plot(column='ant_brannst', legend=True);

To plot two datasets together, we simply need to supply an `ax` (axis) parameter. The axis is also returned by the `plot` function so this is easy:

In [ ]:
ax = fylker.plot(column='ant_brannst', legend=True)
fire.to_crs(fylker.crs).plot(ax=ax, alpha=0.3, color='black');

Before leaving `geopandas`, we'll note that it also supports many operations on the different geometries (we've briefly seen `within`), see the [`geopandas` reference](http://geopandas.readthedocs.io/en/latest/reference.html) for details. As a simple example, we can take the union of the region-polygons to get a polygon of Norway:

In [ ]:
norge_poly = fylker.unary_union
norge_poly

## Using `folium` and `leaflet` to create web maps

[Leaflet](http://leafletjs.com/) is a JavaScript library for displaying web maps similar to Google Maps, OpenStreetMap and Norgeskart. The `folium`-package provides a Python wrapper to Leaflet. It can be installed through `conda-forge`:

    conda install -c conda-forge folium
    
Displaying a map using `folium` is close to trivial:

In [ ]:
import folium

folium.Map([59.9, 10.7], zoom_start=10) #, tiles='Stamen Watercolor')

BY default, `folium` uses OpenStreetMap map tiles. We can also use different map tiles. Several are included in `folium` like the artistic `Stamen Watercolor`-tiles (see [github](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles) for a list). It is also possible to provide other tiles if you know the URL to a tile (WMF) server. The following uses the `topo2`-maps from the Norwegian Mapping Authority:

In [ ]:
map = folium.Map([59.9, 10.7], zoom_start=10,
                 tiles='http://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo2&zoom={z}&x={x}&y={y}',
                 attr='&copy; <a href="http://www.kartverket.no/Kart/Gratis-kartdata/Lisens/">Kartverket</a>',
)
map

As you see, Jupyter Notebooks support displaying `folium` maps directly. If you are using `folium` in an application, you probably want to do something like

    map.save(file_path)
    
which saves the map to an HTML-file which you can open in a browser.

### Oslo bysykkel

The Oslo City Bike project makes available some of their data at [developer.oslobysykkel.no](https://developer.oslobysykkel.no/). Let us have a look at the stations that are available and plot them on a map.

In [ ]:
import json

with open('data/oslobysykkel_stations.json') as fid:
    bike_stations = json.load(fid)
    
bike_stations

Leaflet (and `folium`) supports many different markers. Below, we use `CircleMarker`s to plot each station with a little bit of information in a popup. We also scale the markers depending on the size of the station.

In [ ]:
import html, math

for station in bike_stations['stations']:
    location = (station['center']['latitude'], station['center']['longitude'])
    popup = '{} ({})'.format(html.escape(station['title']), station['number_of_locks'])
    folium.CircleMarker(location, popup=popup, fill=True,
                        radius=math.sqrt(station['number_of_locks']) * 2).add_to(map)
    
map

See the [Leaflet](http://leafletjs.com/) and [Folium](http://python-visualization.github.io/folium/docs-master/) web pages for more documentation.